In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [107]:
model = pyo.ConcreteModel()
model.Xi = pyo.Var(range(1,6), within=Integers,bounds=(0,None))
model.Xi.pprint()

Xi : Size=5, Index=Xi_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  None :  None : False :  True : Integers
      2 :     0 :  None :  None : False :  True : Integers
      3 :     0 :  None :  None : False :  True : Integers
      4 :     0 :  None :  None : False :  True : Integers
      5 :     0 :  None :  None : False :  True : Integers


In [108]:
Xi=model.Xi

In [109]:
model.y = pyo.Var(bounds=(0,None))
model.y.pprint()

y : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  None :  None : False :  True :  Reals


In [110]:
y=model.y

## Constraints

In [111]:
print(sum(Xi[i] for i in range(1,6)))

Xi[1] + Xi[2] + Xi[3] + Xi[4] + Xi[5]


In [112]:
xi_sum=sum(Xi[i] for i in range(1,6))
model.const1=pyo.Constraint(expr=xi_sum+y<=20)

In [113]:
model.const1.pprint()

const1 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                      : Upper : Active
    None :  -Inf : Xi[1] + Xi[2] + Xi[3] + Xi[4] + Xi[5] + y :  20.0 :   True


## Constraint 2

In [114]:
model.const2=pyo.ConstraintList()

In [115]:

for i in range(1,6):
    model.const2.add(expr=Xi[i]+y>= 15)

In [116]:
model.const2.pprint()

const2 : Size=5, Index=const2_index, Active=True
    Key : Lower : Body      : Upper : Active
      1 :  15.0 : Xi[1] + y :  +Inf :   True
      2 :  15.0 : Xi[2] + y :  +Inf :   True
      3 :  15.0 : Xi[3] + y :  +Inf :   True
      4 :  15.0 : Xi[4] + y :  +Inf :   True
      5 :  15.0 : Xi[5] + y :  +Inf :   True


## Constraint 3


In [117]:
print(sum(i*Xi[i] for i in range(1,6)))

Xi[1] + 2*Xi[2] + 3*Xi[3] + 4*Xi[4] + 5*Xi[5]


In [118]:
model.const3=pyo.Constraint(expr= sum(i*Xi[i] for i in range(1,6)) >= 10)

In [119]:
model.const3.pprint()

const3 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                          : Upper : Active
    None :  10.0 : Xi[1] + 2*Xi[2] + 3*Xi[3] + 4*Xi[4] + 5*Xi[5] :  +Inf :   True


## Constraint 4

In [120]:
model.const4=pyo.Constraint(expr=Xi[5]+(2*y)>=30)

In [121]:
model.const4.pprint()

const4 : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  30.0 : Xi[5] + 2*y :  +Inf :   True


## Objective Function

In [122]:
model.obj=pyo.Objective(expr=sum(Xi[i] for i in range(1,6))+y)# by default it is minimize

In [123]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : Xi[1] + Xi[2] + Xi[3] + Xi[4] + Xi[5] + y


In [124]:
model.pprint()# print all formulations

2 Set Declarations
    Xi_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    const2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

2 Var Declarations
    Xi : Size=5, Index=Xi_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : Integers
          2 :     0 :  None :  None : False :  True : Integers
          3 :     0 :  None :  None : False :  True : Integers
          4 :     0 :  None :  None : False :  True : Integers
          5 :     0 :  None :  None : False :  True : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : 

## Define Solver

In [125]:
opt = SolverFactory('glpk')
results = opt.solve(model)

In [126]:
print(pyo.value(Xi[5]))

2.0


In [127]:
print(pyo.value(y))

15.0


In [128]:
for i in range(1,6):
    print(pyo.value(Xi[i]))

0.0
0.0
0.0
0.0
2.0


In [129]:
print('X values are {}'.format([pyo.value(Xi[i]) for i in range(1,6) ]))
print('y value is {}'.format(pyo.value(y)))

X values are [0.0, 0.0, 0.0, 0.0, 2.0]
y value is 15.0


In [130]:
print('Obj = ', pyo.value(model.obj))

Obj =  17.0
